<a href="https://colab.research.google.com/github/cannin/gsoc_2023_pytorch_pathway_commons/blob/main/Baseline_model_with_brca_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Data and Libraries

In [3]:
import pandas as pd
import numpy as np
import sklearn

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type != 'cuda':
    raise SystemError('GPU device not found')

Using device: cuda


In [5]:
X_train = pd.read_csv("/content/drive/MyDrive/gsoc data/brca_tcga_2018/X_train.csv", index_col=0)
y_train = np.loadtxt("/content/drive/MyDrive/gsoc data/brca_tcga_2018/y_train.csv", delimiter=',')

X_test = pd.read_csv("/content/drive/MyDrive/gsoc data/brca_tcga_2018/X_test.csv", index_col=0)
y_test = np.loadtxt("/content/drive/MyDrive/gsoc data/brca_tcga_2018/y_test.csv", delimiter=',')

X_val = pd.read_csv("/content/drive/MyDrive/gsoc data/brca_tcga_2018/X_val.csv", index_col=0)
y_val = np.loadtxt("/content/drive/MyDrive/gsoc data/brca_tcga_2018/y_val.csv", delimiter=',')

In [6]:
X_train.head()

,A1CF,A2M,A4GNT,AAAS,AACS,AADAT,AAK1,AANAT,AARS,AARS2,...,ZP2,ZP3,ZP4,ZRANB1,ZRSR2,ZSCAN10,ZW10,ZWILCH,ZWINT,ZZZ3
TCGA-3C-AAAU-01,0.0000,5798.37,8.6165,649.337,1087.400,5.5145,517.333,0.3447,4409.56,689.897,...,0.0000,411.791,0.0000,879.225,421.862,2.7573,430.824,415.824,931.957,1180.460
TCGA-3C-AALI-01,0.0000,7571.98,0.5438,887.983,836.324,24.4698,848.287,2.1751,4570.96,472.501,...,4.3502,439.222,0.5438,566.068,349.647,0.0000,340.402,1161.330,965.198,406.743
TCGA-3C-AALJ-01,0.9066,8840.40,0.0000,723.481,1533.090,53.4905,568.450,1.8132,2556.66,439.710,...,0.0000,334.506,0.0000,1125.110,470.535,0.0000,304.624,305.422,2531.280,509.519
TCGA-3C-AALK-01,0.0000,10960.20,0.0000,776.996,858.088,57.0956,727.762,0.4137,2122.88,431.936,...,1.6549,127.017,0.4137,780.306,364.501,0.8275,314.026,394.506,668.597,700.869
TCGA-4H-AAAK-01,0.4255,9585.44,0.4255,819.149,840.000,62.9787,678.298,0.4255,2360.85,577.847,...,0.0000,327.979,0.4255,1118.720,305.106,0.0000,297.872,405.711,674.468,881.702


# Modelling

In [7]:
!pip install flaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.5/260.5 kB 4.8 MB/s eta 0:00:00


In [8]:
from flaml import AutoML
import pickle

In [11]:
# Initialize an AutoML instance
automl = AutoML()
# Specify automl goal and constraint
automl_settings = {
    "time_budget": 300,  # in seconds
    "metric": 'mse',
    "task": 'regression',
}
# Train with labeled input data
automl.fit(X_train=X_train, y_train=y_train,
           **automl_settings)

[flaml.automl.logger: 07-28 14:59:28] {1693} INFO - task = regression
[flaml.automl.logger: 07-28 14:59:28] {1700} INFO - Data split method: uniform
[flaml.automl.logger: 07-28 14:59:28] {1703} INFO - Evaluation method: holdout
[flaml.automl.logger: 07-28 14:59:29] {1801} INFO - Minimizing error metric: mse
[flaml.automl.logger: 07-28 14:59:29] {1911} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl.logger: 07-28 14:59:29] {2221} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 07-28 14:59:31] {2347} INFO - Estimated sufficient time budget=25529s. Estimated necessary time budget=180s.
[flaml.automl.logger: 07-28 14:59:31] {2394} INFO -  at 129.8s,	estimator lgbm's best error=1154.2739,	best estimator lgbm's best error=1154.2739
[flaml.automl.logger: 07-28 14:59:31] {2221} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 07-28 14:59:34] {2394} INFO -  at 133.1s,	estimator lgbm's best error=1138.

In [12]:
'''retrieve best config and best learner'''
print('Best ML leaner:', automl.best_estimator)
print('Best hyperparmeter config:', automl.best_config)
print('Best loss on validation data: {0:.4g}'.format(automl.best_loss))
print('Training duration of best run: {0:.4g} s'.format(automl.best_config_train_time))

Best ML leaner: extra_tree
Best hyperparmeter config: {'n_estimators': 14, 'max_features': 0.8959762898635182, 'max_leaves': 11}
Best loss on validation data: 1004
Training duration of best run: 3.316 s


In [13]:
'''pickle and save the automl object'''
import pickle

with open('automl.pkl', 'wb') as f:
    pickle.dump(automl, f, pickle.HIGHEST_PROTOCOL)
'''load pickled automl object'''
with open('automl.pkl', 'rb') as f:
    automl = pickle.load(f)

In [14]:
y_pred = automl.predict(X_test)

In [15]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print('mse', '=', mse)
print('mae', '=', mae)

mse = 1225.7043125692337
mae = 28.01163683512182
